In [30]:
from abc_utils import *
import pandas as pd
from hmmlearn import hmm
import numpy as np
import music21 as m21

In [31]:
train_set, test_set = load_harmonization_train_test()

In [32]:
song_text = train_set.iloc[0]['output']
print(song_text)

T:Train tune 518249
X:518249
E:8
L:1/8
M:3/4
K:C
EF |"C" Ge e2 dc |"G7" c2 B2 DE | FA A2 GF |"C" F2 E2 EF | Gg g2 ge |"Dm" ed d2 cA | 
"G7" AG ^FG ed |"C" c4 EE |"G7" EF F2 ED |"C" E2 C2 GG |"F" FA A2 GF |"C" F2 E2 EF | Gg g2 fe | 
"Dm" ed d2 cA |"G7" AG ^FG ed |"C" c4 GF | EG ce g2 |"G7" f2 dG d2 | FG Bd f2 |"C" e2 cE G2 | 
CE Gc eg |"F" a2 gA c2 |"G7" Bd fG ed |"C" c4 z2 |]


In [34]:
abc_handler = m21.abcFormat.ABCHandler()
abc_handler.tokenize(song_text)
abc_handler.tokenProcess()

for token in abc_handler.tokens:
    print()










_DOC_ATTR
_DOC_ORDER
__annotations__
__class__
__delattr__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__getstate__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__setstate__
__sizeof__
__slots__
__str__
__subclasshook__
_classSetCacheDict
_classTupleCacheDict
_getSlotsRecursive
_reprInternal
_splitChordSymbols
accidentalDisplayStatus
activeDefaultQuarterLength
activeKeySignature
activeTuplet
applicableSpanners
articulations
brokenRhythmMarker
carriedAccidental
chordSymbols
classSet
classes
getPitchName
getQuarterLength
inBar
inBeam
inGrace
isClassOrSubclass
isRest
parse
pitchName
preParse
quarterLength
src
stripComment
tie


In [25]:
def load_song(index):
    start_pos = train_lengths.iloc[:index].sum().item()
    return train_set.iloc[start_pos : start_pos + train_lengths.iloc[index].item()]

In [26]:
song = train_set.iloc[10]['output']

In [29]:
song_df = abc_to_dataframe(song)

In [4]:
def load_songs(indices):
    end_positions = np.cumsum(train_lengths)
    positions = np.insert(end_positions, 0, np.array([0]))
    songs = []
    for i in indices:
        song = train_set.iloc[positions[i] : positions[i+1]]
        songs.append(song)
    # return songs
    return pd.concat(songs)

song_indices = list(range(10))
songs = load_songs(song_indices)

In [5]:
states, observations, times = dataframe_to_states(
    songs, 
    2, 
    1
)

Processing states:   0%|          | 0/944 [00:00<?, ?it/s]

Processing states: 100%|██████████| 944/944 [00:00<00:00, 14851.05it/s]


In [22]:
for row in times:
    print(row)

[0. 0. 1.]
[0.  0.  2.5]
[0. 0. 3.]
[0.  0.  3.5]
[0. 1. 1.]
[0. 1. 2.]
[0.  1.  3.5]
[0. 2. 1.]
[0.  2.  1.5]
[0. 2. 2.]
[0.  2.  2.5]
[0. 2. 3.]
[0.  2.  3.5]
[0. 3. 1.]
[0.  3.  1.5]
[0. 3. 2.]
[0.  3.  2.5]
[0. 3. 3.]
[0.  3.  3.5]
[0. 4. 1.]
[0.  4.  2.5]
[0. 4. 3.]
[0.  4.  3.5]
[0. 5. 1.]
[0. 5. 2.]
[0.  5.  3.5]
[0. 6. 1.]
[0.  6.  1.5]
[0. 6. 2.]
[0.  6.  2.5]
[0. 6. 3.]
[0.  6.  3.5]
[0. 7. 1.]
[0.  7.  2.5]
[0. 7. 3.]
[0.  7.  3.5]
[0. 8. 1.]
[0. 8. 2.]
[0.  8.  3.5]
[0. 9. 1.]
[0. 9. 2.]
[0.  9.  3.5]
[ 0. 10.  1.]
[ 0.  10.   1.5]
[ 0. 10.  2.]
[ 0.  10.   2.5]
[ 0. 10.  3.]
[ 0.  10.   3.5]
[ 0. 11.  1.]
[ 0.  11.   1.5]
[ 0. 11.  2.]
[ 0.  11.   2.5]
[ 0. 11.  3.]
[ 0.  11.   3.5]
[ 0. 12.  1.]
[ 0. 12.  2.]
[ 0.  12.   3.5]
[ 0. 13.  1.]
[ 0. 13.  2.]
[ 0.  13.   3.5]
[ 0. 14.  1.]
[ 0.  14.   1.5]
[ 0. 14.  2.]
[ 0.  14.   2.5]
[ 0. 14.  3.]
[ 0.  14.   3.5]
[ 0. 15.  1.]
[ 0.  15.   2.5]
[ 0. 15.  3.]
[ 0.  15.   3.5]
[ 0. 16.  1.]
[ 0. 17.  1.]
[ 0.  17.   2.5]
[ 0. 

In [6]:
transition_matrix, emission_probs, unique_states, unique_obs, states_to_index, observation_to_index = states_to_transition(states, observations)

In [7]:
emission_probs.shape

(24, 194)

In [8]:
observation_to_index

{(np.float64(0.0),): 0,
 (np.float64(60.0),): 1,
 (np.float64(62.0),): 2,
 (np.float64(64.0),): 3,
 (np.float64(65.0),): 4,
 (np.float64(67.0),): 5,
 (np.float64(68.0),): 6,
 (np.float64(69.0),): 7,
 (np.float64(70.0),): 8,
 (np.float64(72.0),): 9,
 (np.float64(73.0),): 10,
 (np.float64(74.0),): 11,
 (np.float64(75.0),): 12,
 (np.float64(76.0),): 13,
 (np.float64(77.0),): 14,
 (np.float64(79.0),): 15,
 (np.float64(80.0),): 16,
 (np.float64(81.0),): 17,
 (np.float64(82.0),): 18,
 (np.float64(84.0),): 19,
 (np.float64(86.0),): 20,
 (np.float64(87.0),): 21,
 (np.float64(89.0),): 22,
 (np.float64(91.0),): 23}

In [9]:
observation_indices = np.array([observation_to_index[(o,)] for o in observations])

In [10]:
train_lengths.min()

lengths    7
dtype: int64

In [11]:
model = hmm.CategoricalHMM(
    n_components=transition_matrix.shape[0],
    n_features=emission_probs.T.shape[1],
    init_params='',
    algorithm='viterbi'
)

# model.fit(observation_indices.reshape(-1, 1))

In [12]:
model.transmat_ = transition_matrix.T
model.emissionprob_ = emission_probs.T
start_probs = np.ones(transition_matrix.shape[0]) / transition_matrix.shape[0]
model.startprob_ = start_probs

In [13]:
model.n_components

194

Other tools:
- dynamax
  - runs on JAX
  - might have a more flexible structure
  - 

In [14]:
model.transmat_.shape

(194, 194)

In [15]:
model.emissionprob_.shape

(194, 24)

In [16]:
train_lengths.iloc[song_indices].to_numpy().flatten()

array([104,  82,  55, 113,  84,  92,  93, 161,  78,  82])

In [17]:
train_lengths.iloc[song_indices].to_numpy().flatten()

array([104,  82,  55, 113,  84,  92,  93, 161,  78,  82])

In [18]:
likelihood, pred_states = model.decode(
    observation_indices.reshape(-1, 1),
    lengths=train_lengths.iloc[song_indices].to_numpy().flatten(),
    algorithm='viterbi'
)
pred_states

array([  0,  22,  20,  19,  53, 191, 188, 186, 164, 165, 167, 165, 164,
       165, 164, 159, 121, 119, 121, 124, 125, 107,  19,  53, 191, 188,
       186, 164, 159, 121, 120, 118, 105,  16,  13,  19,  35,  83,  86,
        72,  25,  22,  52, 169, 168, 167, 165, 164, 157, 116, 117, 118,
       119, 121, 110,  83,  86,  72,  25,  22,  46, 123, 122, 105,  41,
       118, 105,  16,  13,  19,  20,  51, 167, 168, 169, 171, 173, 174,
       175, 172,  81,  83,  84,  70,  22,  25,  24,  25,  22,  44, 121,
       122, 104,  18,  19,  23,  22,  46, 123, 105,  41, 118, 119, 121,
        98,   2,   8,   9,  10,  11,  10,  12,  14,  15,  14,   8,   9,
        10,  11,  49, 162, 155, 112, 100,   8,   9,  10,  11,  12,  11,
        39, 114,  97,   4,  18,  31,  77,  79,  90, 117, 115, 102,  12,
        14,  15,  14,  15,  14,  14,  37,  82,  80,  79,  78,  68,  14,
        15,  14,  14,  15,  14,  14,  15,  14,  15,  14,  15,  14,  15,
        14,  18,  19,  36,  80,  79,  78,  68,  14,  15,  14,  1

In [19]:
observation_indices.reshape(-1, 1)

array([[18],
       [16],
       [15],
       [14],
       [15],
       [12],
       [12],
       [11],
       [12],
       [14],
       [ 8],
       [11],
       [12],
       [14],
       [12],
       [14],
       [15],
       [14],
       [15],
       [18],
       [16],
       [15],
       [14],
       [15],
       [12],
       [12],
       [11],
       [12],
       [14],
       [ 8],
       [11],
       [14],
       [12],
       [14],
       [15],
       [16],
       [18],
       [21],
       [20],
       [21],
       [18],
       [18],
       [16],
       [15],
       [14],
       [12],
       [11],
       [ 9],
       [ 8],
       [ 9],
       [11],
       [12],
       [14],
       [15],
       [18],
       [21],
       [20],
       [21],
       [18],
       [18],
       [16],
       [15],
       [14],
       [12],
       [11],
       [14],
       [12],
       [14],
       [15],
       [16],
       [12],
       [14],
       [15],
       [16],
       [14],
       [15],
       [16],

In [20]:
unique_states[160]

array([15,  8, 77])

You have state index `i`
- Use `unique_states[i]` to get state as a vector, which contains the last few chords, and the last few melody notes
- `CHORD_INT_MAP` from abc_utils to convert chord indices to chord symbols